In [ ]:
!pip install wheel setuptools pip --upgrade
!pip install --upgrade openai

In [ ]:
import pandas as pd
import json
import random
from openai import OpenAI
import time
import numpy as np
import gzip
import re

API_KEY = ''
client = OpenAI(api_key = API_KEY)
model_id = 'gpt-3.5-turbo-0125' #gpt-3.5-turbo-0125, gpt-4-turbo-2024-04-09

# Replace 'path_to_file.json' with the path to your JSON file
file_path = 'BB_strategyQA.json'

# Open the file and read the JSON data
with open(file_path, 'r') as file:
    data = json.load(file)

# Now 'data' is a Python dictionary containing the data from the JSON file
print(len(data['examples']))

no_count = 0 # 1219
for i in range(len(data['examples'])):
    print(data['examples'][i]['input'])
    print(data['examples'][i]['target_scores']['Yes']) # if 0 --> No

    if data['examples'][i]['target_scores']['Yes'] == 0:
        no_count += 1
    print()


In [ ]:
APE_prompt = f"""
You are given a boolean question statement that requires multi-step implicit reasoning in order to answer correctly. Your task is to propose a creative,
detailed, and step-by-step algorithm to enrich and then reformulate this question statement. The goal is of the algorithm
is to perform a thorough data engineering and reformulation on the statement, so that it is easier for a LLM to generate the answer of the question.
Below are some example question statements as references:

Examples:
- {data['examples'][0]['input'] }
- {data['examples'][1]['input'] }
- {data['examples'][2]['input'] }

For each step of the algorithm, number and then start it on a new line; you must start each step with '###------- Step 1: ' The proposed algorithm will later be submitted to a LLM for execution.
Important: Do NOT answer the question statement! Do NOT refer to any external database; Do NOT perform vector generations. ONLY propose things that a LLM can do on its own!
"""

prompts = []
prompt_performances = {}
for i in range(1):

    completion = client.chat.completions.create(
        model = model_id, temperature = 1.0,

        messages=[{"role": "system", "content": "Please come up with a step-by-step, very specific and detailed, clear, and novel algorithm for reformulating and enriching the question statement. Return the steps ONLY!!! NO more than 3 steps. You MUST NOT directly include answer the question!"},
                    {"role": "user", "content": APE_prompt}],
        timeout = 1200)

    candidate_prompt = completion.choices[0].message.content
    prompts.append(candidate_prompt)
    print(candidate_prompt)
    print()

###------- Step 1: Identify the keywords and entities in the question statement.
###------- Step 2: Break down the question into sub-questions or statements to clarify the context and relationships between the entities.
###------- Step 3: Identify any implied information or assumptions that would be necessary to answer the question accurately.



In [ ]:
import re

# Regular expression to split the text into individual algorithms
algorithm_pattern = r"###------- Step \d: [\w\s]+"

# Split the text based on the pattern
split_text = re.split(algorithm_pattern, candidate_prompt)

# Extract the algorithm headers (for identification)
headers = re.findall(algorithm_pattern, candidate_prompt)

# Removing the first empty string from the split if exists (because of the split at the start)
split_text = [t.strip() for t in split_text if t.strip()]

# Create a dictionary where each algorithm is stored separately
algorithms = {headers[i]: split_text[i] for i in range(len(headers))}

# Display each algorithm separately
steps = []
for header, content in algorithms.items():
    steps.append( [header, content] )
    print(f"{header}:\n{content}\n")

###------- Step 1: Identify the keywords and entities in the question statement:
.

###------- Step 2: Break down the question into sub:
-questions or statements to clarify the context and relationships between the entities.

###------- Step 3: Identify any implied information or assumptions that would be necessary to answer the question accurately:
.



In [ ]:
for p in range(0, len(prompts)):
    # print(candidate_prompt)
    # print()

    total_right = 0
    ADE_total_right = 0
    total = 0

    for i in range(50, len(data['examples'])):
        question = data['examples'][i]['input']
        target = data['examples'][i]['target_scores']['Yes'] # if 0 --> No
        if target == 0:
            target = 'The answer is No' #'No'
        else:
            target = 'The answer is Yes' #'Yes'


        original_prompt = (
            "Given a logic reasoning question, to solve it, you MUST: generate a procedual reasoning from the question statement, and then return the final answer as either 'The answer is Yes' or 'The answer is No'." +
            f'\n\nQuestion: {question}'
        )

        completion = client.chat.completions.create(
            model = model_id, temperature = 0, seed = 0,

            messages=[{"role": "system", "content": "Please solve the logic reasoning question presented by outputting a step-by-step reasoning and then the final answer as either 'The answer is Yes' or 'The answer is No'."},
                        {"role": "user", "content": original_prompt}],
            timeout = 1200)

        ori_answer = completion.choices[0].message.content

        total += 1
        if target in ori_answer:
            total_right += 1


        overall_with_steps = "Original question statement: " + question + '\n\n'
        for i in range(len(algorithms)):

            reformulation_prompt = f'Please thoroughly execute the instruction on the question statement. Instruction:\n\n{steps[i][0]}\n{steps[i][1]}\n\n Question statement: {overall_with_steps}.'

            completion = client.chat.completions.create(
                    model = model_id, temperature = 0.0, max_tokens = 512,

                    messages=[{"role": "system", "content": 'Please execute the instruction on the question statement to be informative and detailed for LLM to interpret better. You are allowed to fill in unspecified information based on your domain expertise!'},
                                {"role": "user", "content": reformulation_prompt}],
                    timeout = 1200)

            reformulated_history = completion.choices[0].message.content
            # print('Step ' + str(i) + ' Results: ')
            # print(reformulated_history)
            overall_with_steps += steps[i][0] + '\n' + steps[i][1] + reformulated_history + '\n\n'
            # print()
            # print()


        # print(overall_with_steps)
        # print()

        ADE_prompt = (
            "Given a logic reasoning question, to solve it, you MUST: generate a procedual reasoning from the question statement, and then return the final answer as either 'The answer is Yes' or 'The answer is No'." +
            f'\n\nQuestion Statment: {overall_with_steps}'
        )

        completion = client.chat.completions.create(
            model = model_id, temperature = 0, seed = 0,

            messages=[{"role": "system", "content": "Please solve the logic reasoning question presented by outputting a step-by-step reasoning and then the final answer as either 'The answer is Yes' or 'The answer is No'."},
                        {"role": "user", "content": ADE_prompt}],
            timeout = 1200)

        answer = completion.choices[0].message.content
        print(ori_answer)
        print(answer)
        print(target)


        if target in answer:
            ADE_total_right += 1

        print(f'Total: {total}, Total Right: {total_right}, Total ADE Right: {ADE_total_right}')
        print()


        if total % 100 == 0:
            print(f'Total: {total}, Total Right: {total_right}, Total ADE Right: {ADE_total_right}')
            print()

            if total_right >= ADE_total_right:
                prompt_performances[candidate_prompt] = ADE_total_right / total
                print()
                break

        if total >= 500:
            prompt_performances[candidate_prompt] = ADE_total_right / total
            break

In [ ]:
# CoT during tuning:
# Total: 168, Total Right: 92, Total ADE Right: 120
# Total: 100, Total Right: 51, Total ADE Right: 63